In [8]:
import os
import mido
import random
from mido import MidiFile, Message
from threading import Thread, Event
from queue import Queue
import simpleaudio as sa
from pathlib import Path

In [9]:
def change_tempo(midi_file_path: str, do_stretch: bool = True) -> str:
    midi = mido.MidiFile(midi_file_path)
    new_tempo = mido.bpm2tempo(80)
    new_message = mido.MetaMessage("set_tempo", tempo=new_tempo, time=0)
    tempo_added = False

    for track in midi.tracks:
        for msg in track:
            if msg.type == "set_tempo":
                track.remove(msg)
                # print(f"removed set tempo message", msg)

        if not tempo_added:
            track.insert(0, new_message)
            # print(f"adding message (tempo=80) {new_message}")
            tempo_added = True

    # if no tracks had a set_tempo message and no new one was added, add a new track with the tempo message
    if not tempo_added:
        new_track = mido.MidiTrack()
        # print(f"adding message to new track {new_message}")
        new_track.append(new_message)

    new_file_path = os.path.join("tmp/playlist", f"{Path(midi_file_path).stem}.mid")
    midi.save(new_file_path)

    return new_file_path

In [10]:
class SimplePlayer:
    do_tick = True

    def __init__(
        self,
        kill_event: Event,
        playback_event: Event,
        filename_queue: Queue,
    ) -> None:
        self.kill_event = kill_event
        self.get_next = playback_event
        self.file_queue = filename_queue

    def playback_loop(self):
        print("sending file request")
        self.get_next.set()
        self.playing_file_path = self.file_queue.get()
        self.playing_file = os.path.basename(self.playing_file_path)
        print(f"got file {self.playing_file}")
        self.get_next.set()

        while not self.kill_event.is_set():
            # get next file from queue
            self.next_file_path = self.file_queue.get()
            next_file = os.path.basename(self.next_file_path)
            self.get_next.set()

            # print progress
            file_tempo = int(os.path.basename(self.playing_file).split("-")[1])
            found_tempo = -1
            for track in MidiFile(self.playing_file_path).tracks:
                for msg in track:
                    if msg.type == "set_tempo":
                        found_tempo = msg.tempo
            print(
                f"playing '{self.playing_file}' ({file_tempo}) --> '{next_file}' ({round(mido.tempo2bpm(found_tempo)):01d})"
            )

            # play file
            self.play_midi_file(self.playing_file_path)

            # playback complete, move to next file
            self.playing_file_path = self.next_file_path
            self.playing_file = os.path.basename(self.playing_file_path)

        print(f"shutting down")

    def play_midi_file(self, midi_path: str) -> None:
        midi = MidiFile(midi_path)

        with mido.open_output("Disklavier") as outport:  # type: ignore
            for msg in midi.play(meta_messages=True):
                if not msg.is_meta:
                    outport.send(msg)
                if msg.type == "text":  # type: ignore
                    self.tick()

                # check if we're dead yet
                if self.kill_event.is_set():
                    self.end_notes()
                    break

    def end_notes(self):
        with mido.open_output("Disklavier") as outport:  # type: ignore
            for note in range(128):
                msg = Message("note_off", note=note, velocity=0, channel=0)
                outport.send(msg)

    def tick(self):
        tick = sa.WaveObject.from_wave_file("../data/tick.wav")
        tick.play()

In [11]:
kill_p1 = Event()
give_p1 = Event()
plst_p1 = Queue()
kill_p2 = Event()
give_p2 = Event()
plst_p2 = Queue()

player1 = SimplePlayer(kill_p1, give_p1, plst_p1)
thread_p1 = Thread(target=player1.playback_loop, args=(), name="p1")
thread_p1.start()

player2 = SimplePlayer(kill_p2, give_p2, plst_p2)
thread_p2 = Thread(target=player2.playback_loop, args=(), name="p2")
thread_p2.start()

dataset_folder = "../data/datasets/careful"

all_segments = os.listdir(dataset_folder)
num_tracks = 0

while num_tracks < 10:
    if give_p1.is_set():
        segment = all_segments[random.randint(0, len(all_segments))]
        seg_path = os.path.join(dataset_folder, segment)
        t_file = change_tempo(seg_path)
        plst_p1.put(t_file)
        num_tracks += 1
        give_p1.clear()

kill_p1.set()
thread_p1.join()

sending file request
sending file request
removed set tempo message MetaMessage('set_tempo', tempo=600000, time=0)
adding message (tempo=80) MetaMessage('set_tempo', tempo=750000, time=0)
got file 20240213-100-03_0312-0320.mid
removed set tempo message MetaMessage('set_tempo', tempo=750000, time=0)
adding message (tempo=80) MetaMessage('set_tempo', tempo=750000, time=0)
playing '20240213-100-03_0312-0320.mid' (100) --> '20231220-80-04_0032-0040.mid' (80)
removed set tempo message MetaMessage('set_tempo', tempo=857142, time=0)
adding message (tempo=80) MetaMessage('set_tempo', tempo=750000, time=0)
playing '20231220-80-04_0032-0040.mid' (80) --> '20240123-70-07_0160-0168.mid' (80)
removed set tempo message MetaMessage('set_tempo', tempo=937499, time=0)
adding message (tempo=80) MetaMessage('set_tempo', tempo=750000, time=0)
playing '20240123-70-07_0160-0168.mid' (70) --> '20240124-64-02_0520-0528.mid' (80)
removed set tempo message MetaMessage('set_tempo', tempo=923076, time=0)
adding m